In [1]:
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline
import numpy as np
from scipy.stats import gaussian_kde

from sklearn.model_selection import train_test_split

# Constants

In [2]:
alpha = 1.0

# Preprocessing

In [3]:
plt.rcParams["figure.figsize"] = (15, 10)
pd.options.display.max_columns = 100

In [4]:
%%time
df = pd.read_csv('data/train_2011_2012_2013.csv', sep=";", nrows=1e10)

<string>:2: DtypeWarning: Columns (17) have mixed types. Specify dtype option on import or set low_memory=False.


CPU times: user 3min 12s, sys: 39.5 s, total: 3min 52s
Wall time: 4min 16s


In [5]:
target = "CSPL_RECEIVED_CALLS"

In [6]:
df.DATE = pd.to_datetime(df.DATE)

In [7]:
import datetime

get the time slot of the day (0 for 00:00, 1 for 00:30, ... 47 for 23:30)

In [8]:
def get_slot(timestamp):
    assert timestamp.minute in [0, 30]
    return 2 * timestamp.hour + (timestamp.minute == 30)

In [37]:
%%time
df['slot'] = df.DATE.apply(get_slot)

CPU times: user 46 s, sys: 2.47 s, total: 48.5 s
Wall time: 48.9 s


In [41]:
%%time
df['dayofweek'] = df.DATE.apply(lambda x: x.dayofweek)

CPU times: user 47.5 s, sys: 3.03 s, total: 50.5 s
Wall time: 52.3 s


In [159]:
%%time
df['ass_hashed'] = df.ASS_ASSIGNMENT.apply(hash)

CPU times: user 4.54 s, sys: 1.28 s, total: 5.82 s
Wall time: 6.49 s


In [31]:
# 1 for night, 0 for day

In [11]:
df.loc[df.TPER_TEAM == 'Nuit', 'TPER_TEAM'] = 0
df.loc[df.TPER_TEAM == 'Jours', 'TPER_TEAM'] = 1

### deleting all columns but very few

In [160]:
df = df[['DATE', target, 'ASS_ASSIGNMENT', 'slot', 'dayofweek', 'ass_hashed']]

### simple model : predict constant value per (ASS_ASSIGNMENT, SLOT, DAYOFWEEK) unique tuple

In [156]:
def best_mean(s):
    """The constant that minimizes the empirical loss.
    For numerical stability, we multiply and divide by the max.
    Equal to 1 / alpha * np.log(1 /s.size * np.sum(np.exp(alpha*s)))"""
    s = alpha * s
    m = s.max()
    s = s - m
    return 1 / alpha * (np.log(1/s.size) + m + np.log(np.sum(np.exp(s))))

In [157]:
def get_best_constant(row):
    
    # selecting corresponding time slots in train set
    sub_df = df[(df.ass_hashed == hash(row.ASS_ASSIGNMENT)) & (df.slot == row.slot) & (df.dayofweek == row.dayofweek)]
#     sub_df = sub_df[sub_df.slot == row.slot]
#     sub_df = sub_df[sub_df.dayofweek == row.dayofweek]
    
    if sub_df.empty:
        print("no data for row={}".format(row))
        return 0

    # get the best prediction constant for that loss
    s = sub_df.reset_index().groupby("DATE")[target].sum()
    return best_mean(s)

In [162]:
%%timeit
get_best_constant(df.iloc[10])

10 loops, best of 3: 129 ms per loop


# Read submission data

In [22]:
sub = pd.read_csv('data/submission.txt', sep='\t')

In [23]:
sub.DATE = pd.to_datetime(sub.DATE)

In [17]:
# there are no training data for the submission dates!
sub_ix = sub.set_index('DATE').index
df_ix = df.set_index("DATE").index
df_ix.intersection(sub_ix)

DatetimeIndex([], dtype='datetime64[ns]', name='DATE', freq=None)

In [24]:
sub.shape

(82909, 3)

In [25]:
sub['dayofweek'] = sub.DATE.apply(lambda x: x.dayofweek)
sub['slot'] = sub.DATE.apply(get_slot)

In [43]:
%%time
get_best_constant(sub.iloc[15])

CPU times: user 503 ms, sys: 12.7 ms, total: 516 ms
Wall time: 515 ms


14.352748662935406

### cell below takes ~3 hours to run

In [85]:
for i in range(sub.shape[0]):
    if not i % 100:
        print("{} %".format(100 * i / sub.shape[0]))
    sub.loc[i, 'prediction'] = get_best_constant(sub.iloc[i])

0.0 %
0.0012061416734009576 %
0.002412283346801915 %
0.003618425020202873 %
0.00482456669360383 %
0.006030708367004789 %
0.007236850040405746 %
0.008442991713806703 %
0.00964913338720766 %
0.01085527506060862 %
0.012061416734009577 %
0.013267558407410535 %
0.014473700080811492 %
0.01567984175421245 %
0.016885983427613407 %
0.018092125101014364 %
0.01929826677441532 %
0.02050440844781628 %
0.02171055012121724 %
0.022916691794618197 %
0.024122833468019154 %
0.025328975141420112 %
0.02653511681482107 %
0.027741258488222027 %
0.028947400161622984 %
0.03015354183502394 %
0.0313596835084249 %
0.03256582518182586 %
0.03377196685522681 %
0.034978108528627774 %
0.03618425020202873 %
0.03739039187542969 %
0.03859653354883064 %
0.039802675222231604 %
0.04100881689563256 %
0.04221495856903352 %
0.04342110024243448 %
0.04462724191583543 %
0.045833383589236394 %
0.04703952526263735 %
0.04824566693603831 %
0.04945180860943926 %
0.050657950282840224 %
0.05186409195624118 %
0.05307023362964214 %
0.0542

In [86]:
sub.prediction.describe()

count    8.290900e+04
mean              inf
std               NaN
min     -1.110223e-16
25%      0.000000e+00
50%      9.135438e-01
75%      1.521440e+01
max               inf
Name: prediction, dtype: float64

In [142]:
sub.head()

,DATE,ASS_ASSIGNMENT,prediction,dayofweek,slot
0,2012-12-28,CMS,0.000000,4,0
1,2012-12-28,Crises,0.000000,4,0
2,2012-12-28,Domicile,0.299646,4,0
3,2012-12-28,Gestion,0.000000,4,0
4,2012-12-28,Gestion - Accueil Telephonique,0.045581,4,0


In [143]:
real_sub = sub.iloc[:, :3]

In [144]:
real_sub.head()

,DATE,ASS_ASSIGNMENT,prediction
0,2012-12-28,CMS,0.000000
1,2012-12-28,Crises,0.000000
2,2012-12-28,Domicile,0.299646
3,2012-12-28,Gestion,0.000000
4,2012-12-28,Gestion - Accueil Telephonique,0.045581


In [145]:
real_sub.DATE = real_sub.DATE.astype(str) + ".000"

In [146]:
real_sub.to_csv('data/sub3.txt', sep='\t', index=None)

In [163]:
# real_sub.prediction = real_sub.prediction.astype(np.int64)